In [1]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
from tqdm import tqdm

from statsmodels.stats.proportion import proportions_ztest, proportions_ztost, test_proportions_2indep
from statsmodels.stats.nonparametric import rank_compare_2indep
from statsmodels.stats.weightstats import ttest_ind, ttost_ind
from scipy.stats import pearsonr, spearmanr, mannwhitneyu, ttest_rel

## Power Analysis

In [2]:
def rand_bin_array(topics, questions=30):
    selections = np.zeros(topics)
    selections[:questions]  = 1
    np.random.shuffle(selections)
    return selections

def permutation_test(a, b, alternative="two-sided", value=0, iters=1000):
    og_diff = np.mean(a) - np.mean(b) - value
    combin = np.concatenate([a, b])
    n_a, n_b = len(a), len(b)
    diffs = [
        np.mean(samp[:n_a]) - np.mean(samp[n_a:])
        for _ in range(iters)
        for samp in [np.random.choice(combin, n_a + n_b, replace=False)]
    ]
    if alternative == "two-sided":
        return og_diff, np.mean(np.abs(og_diff) < np.abs(diffs)), diffs
    elif alternative == "larger":
        return og_diff, np.mean(og_diff < diffs), diffs
    elif alternative == "smaller":
        return og_diff, np.mean(og_diff > diffs), diffs
    else:
        raise ValueError("alternative must be one of (two-sided, larger, smaller)")

### intrusion power analysis

In [204]:
# params
topics = 50
annotators_per_topic = 26
prob_correct = np.array([
    1/6, # how often someone selects the intruder for an incoherent topic
    0.85, # how often someone selects the intruder for a coherent topic. estimated from chang 2009: avg. % correct of the 10% most-correct topics
])
bad_topic_difference = 4 # effect size (number of bad topics in model a vs model b)
h0 = 0.0

# simulation
iter = 10_000
pvals = []
alternative = "larger" if h0 == 0 else "smaller"
np.random.seed(454)

for i in range(iter):
    # create topics for both models
    model_a_topics = np.random.choice([0, 1], topics)
    if model_a_topics.sum() - bad_topic_difference < 0:
        continue
    model_b_topics = np.zeros(topics, dtype=int)
    model_b_topics[:int(model_a_topics.sum() - bad_topic_difference)] = 1

    # simulate responses
    n = topics*annotators_per_topic
    topic_assignments = annotators_per_topic
    prob_a = prob_correct[model_a_topics]
    model_a_responses = np.random.binomial(n=topic_assignments, p=prob_a)
    prob_b = prob_correct[model_b_topics]
    model_b_responses = np.random.binomial(n=topic_assignments, p=prob_b)
    
    # do test
    #pval, _, _ = proportions_ztost([model_a_responses.sum(), model_b_responses.sum()], [n, n], low=-5, upp=h0)
    stat, pval = proportions_ztest([model_a_responses.sum(), model_b_responses.sum()], [n, n], value=h0, alternative=alternative)
    #stat, pval = test_proportions_2indep(model_a_responses.sum(), n, model_b_responses.sum(), n, alternative='larger', compare='diff')
    pvals.append(pval)
pvals = np.array(pvals)

In [205]:
for alpha in [0.01, 0.025, 0.05]:
    print(f"alpha {alpha:5}, power: {np.mean(pvals < alpha):0.3f}")

alpha  0.01, power: 0.737
alpha 0.025, power: 0.877
alpha  0.05, power: 0.946


### ratings power analysis

In [226]:
# params
topics = 50
annotators_per_topic = 15
# probabilities of selecting bad, okay, good when 
# the "true" topic is bad, okay, or good
topic_probs = np.array([
    [0.75, 0.25, 0.0], # bad topic probs
    [0.25, 0.5, 0.25], # okay topic probs
    [0.0, 0.25, 0.75], # good topic probs
])
bad_topic_difference = 4 # effect size (number of bad topics in model a vs model b)
h0 = 0.0 # null value. Set to >0 if `bad_topic_difference` is 0

# simulation
iter = 10_000
pvals_rho, pvals_t, pvals_u = [], [], []
alternative = "larger" if h0 == 0 else "smaller"

for i in tqdm(range(iter), total=iter):
    # create topics for both models
    model_a_topics = np.random.choice([0, 1, 2], topics)
    good_topics_in_a = (model_a_topics == 2).sum()
    okay_topics_in_a = (model_a_topics == 1).sum()
    good_topics_in_b = good_topics_in_a - bad_topic_difference
    if good_topics_in_b < 0:
        continue
    model_b_topics = np.zeros(topics, dtype=int)
    model_b_topics[:good_topics_in_b] = 2
    model_b_topics[good_topics_in_b:okay_topics_in_a+good_topics_in_b] = 1
    assert((model_b_topics == 1).sum() == okay_topics_in_a)

    # simulate responses
    n = topics*annotators_per_topic

    prob_a = topic_probs[model_a_topics]
    model_a_responses = np.array([
        np.random.multinomial(annotators_per_topic, p) for p in prob_a
    ])
    model_a_responses = np.repeat([0, 1, 2], model_a_responses.sum(0))
    prob_b = topic_probs[model_b_topics]
    model_b_responses = np.array([
        np.random.multinomial(annotators_per_topic, p) for p in prob_b
    ])
    model_b_responses = np.repeat([0, 1, 2], model_b_responses.sum(0))
    # do test
    # t-test
    stat, pval, dof = ttest_ind(
        model_a_responses,
        model_b_responses,
        value=h0,
        alternative=alternative,
    )
    pvals_t.append(pval)
    # https://www.uvm.edu/~statdhtx/StatPages/More_Stuff/OrdinalChisq/OrdinalChiSq.html
    # "ordinal" chi-sq
    if h0 == 0:
        stat, pval = pearsonr(
            np.concatenate([model_b_responses, model_a_responses]),
            np.concatenate([np.zeros(n), np.ones(n)])
        )
        pvals_rho.append(pval)
    # mann-whitney
    if h0 == 0:
        stat, pval = mannwhitneyu(
            model_a_responses,
            model_b_responses,
            alternative="greater",
        )
        pvals_u.append(pval)
pvals_rho = np.array(pvals_rho)
pvals_t = np.array(pvals_t)
pvals_u = np.array(pvals_u)


100%|██████████| 10/10 [00:00<00:00, 311.28it/s]


In [203]:
for alpha in [0.01, 0.025, 0.05]:
    print(
        f"alpha: {alpha}\n",
        f"   power for corr.: {np.mean(pvals_rho < alpha):0.3f},",
        f"power for ttest: {np.mean(pvals_t < alpha):0.3f},",
        f"power for m-w: {np.mean(pvals_u < alpha):0.3f}",
    )

alpha: 0.01
    power for corr.: nan, power for ttest: 0.753, power for m-w: nan
alpha: 0.025
    power for corr.: nan, power for ttest: 0.891, power for m-w: nan
alpha: 0.05
    power for corr.: nan, power for ttest: 0.961, power for m-w: nan
/workspace/.conda/envs/topic-preprocessing/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/workspace/.conda/envs/topic-preprocessing/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### auto power analysis

In [9]:
# params
topics = 50
mean = 0.5
# npmi
sd_gamma_params = 15, 0.005 # roughly lines up with empirical
# c_v
#sd_gamma_params = 20, 0.005

bad_topic_difference = 7 # effect size (number of bad topics in model a vs model b)
h0 = 0.0

# simulation
iter = 10_000
pvals = []
alternative = "larger" if h0 == 0 else "smaller"

for i in tqdm(range(iter), total=iter):
    # create topics for both models
    sd = max(0.0001, np.random.gamma(*sd_gamma_params))
    model_a_scores = np.random.normal(mean, sd, size=topics)
    # model b is like model a but...
    model_b_scores = np.sort(np.random.normal(mean, sd, size=topics))
    # the highest-scoring topics are replaced with the lowest scoring, plus some noise
    if bad_topic_difference > 0:
      model_b_scores[topics - bad_topic_difference:] = np.random.normal(model_b_scores[0], sd, size=bad_topic_difference)
    
    # do test
    #stat, pval = rank_compare_2indep(model_a_scores, model_b_scores)
   # stat, pval, diffs = permutation_test(model_a_scores, model_b_scores, value=h0, alternative=alternative, iters=500)
    stat, pval, dof = ttest_ind(model_a_scores, model_b_scores, value=h0, alternative=alternative)
    # stat, pval = mannwhitneyu(model_a_scores, model_b_scores, alternative='greater')
    pvals.append(pval)
pvals = np.array(pvals)

100%|██████████| 10000/10000 [00:03<00:00, 3316.11it/s]


In [10]:
for alpha in [0.01, 0.025, 0.05]:
    print(f"alpha {alpha:5}, power: {np.mean(pvals < alpha):0.3f}")

alpha  0.01, power: 0.578
alpha 0.025, power: 0.717
alpha  0.05, power: 0.819
